1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



#Использование логарифма как функции дисконтирования можно объяснить следующими интуитивными соображениями: с точки зрения ранжирования позиции в начале списка отличаются гораздо сильнее, чем позиции в его конце. Так, в случае поискового движка между позициями 1 и 11 целая пропасть (лишь в нескольких случаях из ста пользователь заходит дальшей первой страницы поисковой выдачи), а между позициями 101 и 111 особой разницы нет — до них мало кто доходит. Эти субъективные соображения прекрасно выражаются с помощью логарифма:

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [14]:
import pandas as pd
import numpy as np
import math


In [21]:
bought_list = [1, 3, 5, 7, 9, 11]
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [22]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

precision_at_k(recommended_list, bought_list, k=8)

0.625

In [23]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

recall_at_k(recommended_list, bought_list, k=8)

0.8333333333333334

In [24]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        print(i)
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            print(p_k)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

ap_k(recommended_list, bought_list, k=8)

1
1.0
2
1.0
3
1.0
4
5
0.8
6
0.8333333333333334
7
8


0.5791666666666666

In [28]:
import math

def dcg_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
#     if sum(flags) == 0:
#         return 0
    
    sum_ = 0
    num = 2
    for i in range(2, k+1): 
#         print(i)
        if flags[i-1] == True:
#             print('yes')
#             print(math.log(i))
            spam = 1/math.log(i+1)
#             print(spam)
            sum_ += spam
        else:
#             print('no')
            spam = 0/math.log(i+1)
#             print(spam)
            sum_ += spam
            
#     print('sum_', sum_)
    
    if flags[0] == True:
        result = 1/k * (1/1 + sum_)
    else:
        result = 1/k * (0/1 + sum_)
    return result

def ideal_dcg_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
#     flags = np.isin(recommended_list, bought_list)
    
    sum_ = 0
    num = 2
    for i in range(2, k+1): 
#         print(i)
        
        spam = 1/math.log(i+1)
#         print(spam)
        sum_ += spam

            
#     print('sum_', sum_)
    result = 1/k * (1/1 + sum_)
    
    return result


dcg_k(recommended_list, bought_list, k=8), ideal_dcg_k(recommended_list, bought_list, k=8)

(0.4629494644990396, 0.6576185763535419)

In [31]:
ndcg_k = dcg_k(recommended_list, bought_list, k=8) / ideal_dcg_k(recommended_list, bought_list, k=8)
ndcg_k

0.7039786909093542

In [19]:
def reciprocal_rank_k(recommended_list, bought_list, k=8):
    
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

reciprocal_rank_k(recommended_list, bought_list, k=8)

0.13

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [4]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5